# Model-fitting Cookbook

## Notes

### Goal

To gain a little familiarity with the basics of fitting models to data, at the level usually seen in upper-division lab courses.

Keep in mind that statistics is properly its own field, and no 1 hour introduction is going to be adequate. The idea that after reading this, looking at a worked example, and working a problem yourself, you'll have learned enough words to sign along, not that you'll necessarily be ready for a solo.

These notes are meant to be a **cookbook**. You'll see recipes for doing things, but not all the math behind _why_ we do those things. An accompanying notebook contains a worked example, and is meant to be read in parallel with this one.

### Introduction

Model fitting shows up very often in experimental/observational astrophysics.

Sometimes the model is physically based, and finding the value of its parameters is the whole point (e.g. learning the expansion rate of the Universe from Cepheid data).

Other times, we just need a simple function to describe some data for our own nefarious purposes. 

To be clear, _model_ here means some set of equations that make predictions for the data (e.g. a line, $y=a+bx$), which may contain parameters ($a$ and $b$) that we either claim to know already, or want to learn from the data.

<table><tr width=90%>
    <td width=45%><a href="graphics/LICENSE.txt"><img src="graphics/fgas_z_lcdm_models.png" width=300></a></td>
</tr></table>

Above: Some real-life data and three potential models for them. Underlying each model is a relatively complex set of equations with different parameters determining the model prediction as a function of redshift.

Here is a subset of the questions we might have, given some data on one hand and a model that is supposed to explain where they come from on the other:
1. What are the best-fitting parameter values given the data?
2. What are our uncertainties on those values?
3. Is the fit any good?
4. Do the data support one model more than another?

#### What you should remember, if nothing else:

* There is a correct way to do statistical inference from data. This is not an exercise in choosing something that kinda seems like it maybe should work.

That said, the _really_ correct approach will be covered in the next statistics session. Everything today is technically approximate, except in special cases. But it's a good place to start, and is extremely widely used regardless.

* The model we fit must include data collection, not just physics.

That is, the job of a model isn't over when a physical source we're observing is described. It needs to make a prediction for the actual numbers that are recorded by our instruments, including any biases or randomness introduced by the measurement process.

Another way of thinking about this is to say that _data do not have errors_. Data are the values that we record, and they are immediately set in stone. Instead, think of models making a _probabilistic prediction_ for what data might have been observed, as a function of the model parameters.

#### About "error bars"

Let's take a very simple example: I want to meaure the flux of a star and have a perfect, background-free detector. In a given exposure time, $t$, a flux $F$ would predict that _on average_ I measure a certain number of photons with my detector, $\mu = Fct$, where the constant $c$ takes care of the conversions between flux and count rate for my specific telescope and detector.

When counting discrete events like this, the actual number we get in an observation, $N$, will be random and given by the Poisson distribution:

$P(N|\mu) = \frac{\mu^N e^{-\mu}}{N!}$

Note that there is no "error bar" on the measurement here. Instead, we have _much more_ information in the form of a probability distribution. Moreover, that distribution is determined by the model (via $\mu$), rather than the data per se.

It so happens that if $\mu$ is Sufficiently Large, the Poisson distribution becomes very well approximated by the normal (or Gaussian) distribution,

$P(N|\mu) \rightarrow \frac{1}{\sqrt{2\pi}\sigma}\exp\left[ -\frac{(N-\mu)^2}{2\sigma^2} \right]$; $\mu >> 1$

with $\sigma\approx\sqrt{\mu}$.

In fact, fairly generically, if enough independent random variables are added together, even if they follow different distributions, the central limit theorem says the result should approximately follow a Gaussian distribution.

So, for quantities that are more complex than the simple case above, it very often happens that we are willing to approximate the associated model predictions as being Gaussian distributed.

The symmetry between data and model in the Gaussian distribution, $(N-\mu)^2$, is why we can get away with thinking about an "error bar" as being associated with the measured value. (Provided that the Gaussian width, $\sigma$, doesn't depend on model parameters, as it also did above.)

It is undoubtedly convenient to be able to center error ranges on each point when plotting things (e.g. in the figure above), but we should always keep in mind that it's the model predictions that are uncertain, not the data.

#### The recipes

For context, the procedures outlined below go under the name **maximum likelihood methods**. These are about the most generally applicable methods available without diving into the Bayesian approach covered in our next session. In the other direction, there are linear and non-linear least-squares methods which are more specialized than maximum-likelihood, but might be appropriate for a given problem (see below; connections to these are also mentioned in the worked example notebook).

You may find it helpful to follow along the analogous parts of the the worked example notebook while reading through the procedures below. In particular, the example notebook generates lots of useful visualizations which are absent from these dry notes.

### The likelihood function

A key concept above was that our model must make a probabilistic prediction for the data we measure, as a function of some parameters. This is expressed mathematically in a likelihood function.

The likelihood function is also sometimes called a "sampling distribution" because it should be a probability distribution over possible data sets, given parameters. But when we think of it as a function of parameters for fixed (observed) data, it goes by the name "likelihood".

We already wrote down the sampling distribution for the simple-minded flux measurement described above:

$P(N|F) = \frac{(Fct)^N e^{-Fct}}{N!}$.

Here I'm assuming $c$ and $t$ are assumed to be known perfectly, so we don't have to list them as model parameters.

The likelihood function is just the same, but conventionally written with a different notation emphasizing that it's a function of the parameter(s):

$\mathcal{L}(F;N) = \frac{(Fct)^N e^{-Fct}}{N!}$.

For convenience, and because it clarifies the link to least squares in the appropriate limit, we actually work with $-2$ times the log-likelihood,

$-2\left[ -Fct +N\ln(Fct) - \ln(N!)\right]$.

It turns out that we don't need to care about the absolute value of this thing, just changes in it as the parameters are varied. So, we could throw away any terms that don't include $F$, and simplify to (in this case)

$S(F;N) = 2Fct -2N\ln(F)$.

Aside: remember that probabilities of independent things are multiplied to get the effect of "and", $P(A~\mathrm{and}~B)=P(A)P(B)$. So, if we have multiple, independent (for given parameter values) data points, we end up with a product over data points in the likelihood, or a sum over data in $S$.

#### Special cases

If the likelihood function is a product of normal (aka Gaussian) distributions with known widths, and the mean of each distribution is a linear function of the model parameters, this is a **Linear Least Squares** problem. The example notebook uses the canonical and familiar linear least squares model (fitting a line to data points in the $x$-$y$ plane), and comments on the connection to the more general maximum-likelihood approach, so we won't say too much more about linear least squares here in the notes.

If the likelihood function is a product of normal (aka Gaussian) distributions with known widths, and the mean of each distribution is a _non-linear_ function of the model parameter, this is (fittingly enough) a **Non-linear Least Squares** problem.

These special cases will be mentioned again below, but for the most part we proceed with methods that apply even when the likelihood is not Gaussian, e.g. the Poisson likelihood above.

### The best fit

The likelihood tells us the probability that a given set of parameter values would produce the measured data. So, we define the best-fitting parameter values as those that maximize the likelihood, or minimize $S$.

In general, we are stuck using an iterative minimizer (e.g. from `scipy.optimize`) to find the best fit. (For linear least squares, there is a linear algebraic solution.)

### Confidence intervals

If finding the best fit is as straightforward as minimizing a function (however straightforward that actually is), then determining uncertainties on parameters is more subtle.

First, a _confidence interval_ a range of values for a single parameter, associated with a given probability (between 0 and 1), called a _confidence level_. One can find analogous _confidence regions_ for 2 or more parameters simultaneously, that consist of areas/volumes in higher dimensional parameter spaces.

Before discussing what they actually mean, which is a can of worms, let's first see how to determine them.

Let's say our model has many parameters: $\theta_0$, $\theta_1$, $\theta_2$, ...

To find a confidence interval for $\theta_0$, we would first construct the _profile likelihood_ for $\theta_0$.

This means, *as a function of* $\theta_0$, the maximum likelihood achievable by varying the other parameters.

$S_{\theta_0}(\theta_0) = \min_{\theta_1,\theta_2,\ldots} S(\theta_0,\theta_1,\theta_2,\ldots;\mathrm{data})$

We would refer to the resulting confidence interval as _marginalized_ over $\theta_1$, $\theta_2$, ... because we allow those parameters to take whatever value maximizes the likelihood, for a given $\theta_0$.

The confidence interval itself is the range of $\theta_0$ values satisfying

$S_{\theta_0}(\theta_0) - S_\mathrm{min} \leq \Delta S_1(\alpha)$,

where $S_\mathrm{min}$ is the absolute minimum of $S$ (i.e., $S$ for the best fit parameters), and $\alpha$ is the confidence level we're looking for.

The subscript "1" on $\Delta S$ reflects the fact that these thresholds depend not just on $\alpha$, but on the number of parameters the confidence interval/region is for. For an interval, this is 1, but for a joint confidence region for 2 parameters, it would be 2, etc.

For 2 parameters, the confidence region would be an area in the (e.g.) $\theta_0$-$\theta_1$ plane where

$S_{\theta_0,\theta_1}(\theta_0,\theta_1) - S_\mathrm{min} \leq \Delta S_2(\alpha)$,

where $S_{\theta_0,\theta_1}(\theta_0,\theta_1) = \min_{\theta_2,\ldots} S(\theta_0,\theta_1,\theta_2,\ldots;\mathrm{data})$

There are tables of these $\Delta S$ values, but you can also compute them:

In [ ]:
import scipy.stats as st
def deltaS_threshold(conf_level, npar)
    return st.chi2.ppf(conf_level, npar)

The call above is to the _quantile function_ of the $\chi^2$ probability distribution. Without worrying about the details, this (the $\chi^2$) is a clue that we are leaning on a result that is exact for linear least squares problems, and only asymptotically correct otherwise. (The example notebook has a bit to say about what distinguishes linear least squares from the more general case we're working with.)



Usually, having a Sufficiently Large number of data points is what gets one into the asymptotic regime where this is a very good approximation. However, there are no guarantees in full generality, at least without doing a ton of careful math. This has never stopped anyone else from taking the leap, so we won't let it stop us either. But again, see the next session for a more complete approach.


Here are the most common confidence levels (columns) and $\Delta S$ thresholds you'll see:

| $n_\mathrm{param}$ | 0.683 | 0.954 | 0.997 |
| :---: | :---: | :---: | :---: |
| 1 | 1.0 | 4.0 | 9.0 |
| 2 | 2.30 | 6.18 | 11.83 |

These seemingly ridiculous confidence levels are conventional, and correspond to the area under the Gaussian distribution integrated between $\pm1$ (and $\pm2$ and $\pm3$) standard deviations of the mean. They are commonly referred to as $1\sigma$, $2\sigma$, $3\sigma$, etc. as a result. These confidence levels can also be computed to silly precision easily:

In [ ]:
def confLevel(Nsigma):
    return st.chi2.cdf(Nsigma**2, 1)

The `cdf` function above (the _cumulative distribution function_) is the inverse of the quantile function, which is why the $n_\mathrm{param}=1$ line of the table was simply $1^2$, $2^2$, $3^2$, ...

One rarely sees joint confidence regions for $>2$ parameters, simply because they are difficult to present and visualize.

A 1-parameter best fit value and confidence interval can be stated easily as, e.g., $\theta_0=3.14^{+0.06}_{-0.09}$. This would normally be interpretted as: 3.14 is the best fitting value, and the ($1\sigma$, if not specified) confidence interval is $[3.05,3.20]$.

A 2-parameter confidence region would normally be presented as a contour in the $\theta_0$-$\theta_1$ plane. Normally, $1\sigma$ and $2\sigma$ regions (at least) are shown.

One more thing to mention is that the math underneath all these $\Delta S$ calculations _also_ assumes that our best fit is, in fact, the true model that the data came from.

The fact that our best-fitting parameter values aren't exactly the true values isn't so bad, but we shouldn't trust the confidence intervals unless the fitted model is a good description of the data (next section).

Finally, some words about what confidence regions actually mean. It would be very tempting to interpret a 68% confidence interval for $\theta_0$ as an interval that contains the true value of $\theta_0$ with 68% probability. **This is not the case**, although I suspect that everyone secretly thinks about them that way.

In fact, a 68% confidence interval as we have defined it means:

> If it were somehow possible to independently repeat the experiment arbitrarily many times, and we computed a 68% confidence interval according to the recipe above for each experiment, then 68% of those intervals would contain the true value of $\theta_0$.

You could be forgiven for having to read that last bit several times. In short, these confidence regions answer a question that exactly no one has asked or been interested in, ever. Yet this is what's very often done.

It _is_ possible to answer the more straightforward question "What is the most probable range of values for $\theta_0$?" using Bayesian analysis (as we'll see next time). And _reasonably often_ we are in the limit where the answer would be the same, or very close. So knowing how to use the methods above is worthwhile, even if the interpretation is odd.

### Goodness of fit

Ok, we now have best-fitting values and confidence ranges for our parameters, but how do we know whether that best fit is a _good_ fit?

One option, whose usefulness _cannot be overstated_ is to plot the predictions of the model over the data in a sensible way, and compare them visually.

The reasoning is that a "good" model fit is one that could plausibly produce the data that we have.

To be more quantitative, we would choose a number that somehow summarizes the data and compare it to the predictions of the best-fitting model. Since the model normally has some randomness in it, one could think about generating multiple mock data sets from it, which would be different even for fixed values of the parameters. For each of these mock data sets, the model that generated them is by definition a good fit. In other words, we would have a "true" summary statistic computed from the data, and a distribution of summary statistics predicted by the best model.

In principle the summary could be anything (e.g. the mean of the data), but usually it's chosen to be the minimum $S$ found by fitting the model parameters to the (real of mock) data.

Recall that for linear least squares problems (and others, when we are willing to hold our noses), $S$ is closely related to the $\chi^2$ distribution. In that limit, we know analytically exactly what the distribution of $S_\mathrm{min}$ from many mock data sets based on a good fit would be.

More generally, we can actually carry out the process of generating mock data and fitting them, and compare the distribution from these simulations with the value of $S_\mathrm{min}$ from real data.

Both of these options are spelled out more in the example notebook.

### Model comparison

If you find that the model you've fit is _not_ good, the next question is how to determine whether an alternative model is better.

A model with more flexibility (i.e. more free parameters) will usually achieve a lower $S_\mathrm{min}$, but this alone doesn't mean it's better. We want to know whether a given reduction in $S_\mathrm{min}$ would be expected due to the increased flexibility, _even if the simpler model were true_. If not, the more flexible model is genuienly a better fit.

Note that this is phrased in terms of _rejecting_ a simple model in favor of a more complex one. The question we ask inherently treats the two options differently by building in the notion of Occam's Razor - if two models are equally good, simpler is better.

The model comparison procedure is similar to the goodness of fit test above for a single model. In this case, we generate many mock data sets from the _simpler_ of the two models. Each mock data set is then used to fit both models, and we record the difference in $S_\mathrm{min}$ between them, to form a distribution of $S_\mathrm{min}$ values expected _assuming the simpler model to be correct_.

If the reduction in $S_\mathrm{min}$ for the real data is larger than most of these simulated values (e.g. 99%), we would conclude that the more complex model is justified by the data.

Once again, the numerically intensive procedure described above is instantaneous for the special case of linear least squares. In that case, one can compute the $F$ statistic, which is a combination of $S_\mathrm{min}$ for the two models, the number of parameters in each model, and the number of data points.

Under the assumptions that make linear least squares exact, $F$ has a known, analytic distribution, and we can immediately compare the $F$ value we compute for the data to it.

### Parting words

The notes above are hopefully enough to get started fitting models to data. Just to reiterate, even though these procedures can seem ad hoc, there are basic principles working behind the scenes here. In the author's highly biased opinion, it's an excellent idea to learn more about the mathematical foundations of statistical data analysis when you have the opportunity.

For now, you're encouraged to look through the example notebook, which works through most of the steps above, and then put them into practice yourself in the exercise notebook.